In [1]:
from selenium import webdriver as wd
import csv
from bs4 import BeautifulSoup as bs

In [2]:
#WebDriver

In [4]:
driver = wd.Chrome(r'Z:\Class Repo\Personal Practice\Web Scraping\chromedriver_win32\chromedriver.exe')
url = 'https://www.amazon.com/'
driver.get(url)

In [5]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=1HT27OCBDLP2O&sprefix=fishtank%2Caps%2C87&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
    return template.format(search_term)


In [6]:
url = get_url('fishtank')
print(url)

https://www.amazon.com/s?k=fishtank&crid=1HT27OCBDLP2O&sprefix=fishtank%2Caps%2C87&ref=nb_sb_noss_2


In [7]:
driver.get(url)

In [8]:
#Extract Collection

soup = bs(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type':'s-search-result'})


In [9]:
len(results)

60

In [10]:
#Prototype Record

item = results[0]
atag = item.h2.a
desc = atag.text.strip()

In [13]:
url = 'https://www.amazon.com/' + atag.get('href')
print(url)

https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A059103012NFR7AT9EL0V&qualifier=1673310915&id=2979853019093835&widgetName=sp_atf&url=%2FTetra-Gallon-Aquarium-Filter-Conditioners%2Fdp%2FB07BJ9KPSR%2Fref%3Dsr_1_1_sspa%3Fcrid%3D1HT27OCBDLP2O%26keywords%3Dfishtank%26qid%3D1673310915%26sprefix%3Dfishtank%252Caps%252C87%26sr%3D8-1-spons%26psc%3D1


In [14]:
price_parent = item.find('span', 'a-price')
print(price_parent)

<span class="a-price" data-a-color="base" data-a-size="xl"><span class="a-offscreen">$365.01</span><span aria-hidden="true"><span class="a-price-symbol">$</span><span class="a-price-whole">365<span class="a-price-decimal">.</span></span><span class="a-price-fraction">01</span></span></span>


In [15]:
price = price_parent.find('span','a-offscreen').text
print(price)

$365.01


In [15]:
rating = item.i.text
print(rating)

4.4 out of 5 stars


In [16]:
review_count = item.find('span', {'class': 'a-size-base','class': 's-underline-text'}).text
print(review_count)


537


In [18]:
#error Handling
def extract_record(item):
    #Desc and URL
    atag = item.h2.a
    desc = atag.text.strip()
    url = 'https://www.amazon.com/' + atag.get('href')
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base','class': 's-underline-text'}).text
    except AttributeError:
        rating = ' '
        review_count = ' '
        
    result = (desc, price, rating, review_count, url)
    return result
extract_record(item)

('Tetra 55 Gallon Aquarium Kit with Fish Tank, Fish Net, Fish Food, Filter, Heater and Water Conditioners',
 '$365.01',
 '4.4 out of 5 stars',
 '537',
 'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A059103012NFR7AT9EL0V&qualifier=1673310915&id=2979853019093835&widgetName=sp_atf&url=%2FTetra-Gallon-Aquarium-Filter-Conditioners%2Fdp%2FB07BJ9KPSR%2Fref%3Dsr_1_1_sspa%3Fcrid%3D1HT27OCBDLP2O%26keywords%3Dfishtank%26qid%3D1673310915%26sprefix%3Dfishtank%252Caps%252C87%26sr%3D8-1-spons%26psc%3D1')

In [19]:
records = []
soup = bs(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type':'s-search-result'})


for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [20]:
records[1]

('Aqueon LED MiniBow Small Aquarium Fish Tank Kit with SmartClean Technology, Black, 2.5 Gallon',
 '$48.99',
 '4.3 out of 5 stars',
 '3,237',
 'https://www.amazon.com//Aqueon-MiniBow-SmartClean-Technology-Gallon/dp/B08LK994Y3/ref=sr_1_2?crid=1HT27OCBDLP2O&keywords=fishtank&qid=1673310915&sprefix=fishtank%2Caps%2C87&sr=8-2')

In [21]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&page{}&crid=1HT27OCBDLP2O&sprefix={}%2Caps%2C87&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
        
    global page_counter
    page_counter=1
    for page_counter in range(1,21):
        page_inserter = str(page_counter)
        url = template.format(search_term, page_inserter, search_term)
        page_counter += 1
        print(page_counter)
    return url